Create matrix.csv from matrix.mtx file

In [16]:
from scipy.io import mmread
from scipy.sparse import save_npz
import numpy as np

path = 'data/sample1/GSE294399_WBC_020823_matrix.mtx/matrix.mtx'
sparse_matrix = mmread(path)
csr = sparse_matrix.tocsr()
#save_npz("data/sample1/csc.npz", csc_matrix)
np.savez("data/sample1/matrix.csr", data=csr.data, indices=csr.indices, indptr=csr.indptr, shape=csr.shape)

Clustering algorithm using a neural network

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from keras.layers import Input, Dense, Layer
from keras.models import Model
from keras import backend as K

# Load data (replace 'cell_gene_data.csv' with your data file)
data = pd.read_csv('data/matrix.csv', index_col=0)
X = data.values

# Preprocessing: Log transform and scaling
X = np.log1p(X)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

input_dim = X_scaled.shape[1]
encoding_dim = 32  # Latent space dimension
n_clusters = 30     # Number of clusters (adjust as needed)

# Step 1: Pretrain Autoencoder
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Train autoencoder
autoencoder.fit(X_scaled, X_scaled, epochs=100, batch_size=32, shuffle=True)

# Extract encoder model
encoder_model = Model(inputs=input_layer, outputs=encoder)
encoded_data = encoder_model.predict(X_scaled)

# Step 2: Initialize Cluster Centers using K-means
kmeans = KMeans(n_clusters=n_clusters, n_init=10)
kmeans.fit(encoded_data)
cluster_centers = kmeans.cluster_centers_

# Step 3: Define Clustering Layer
class ClusteringLayer(Layer):
    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights

    def build(self, input_shape):
        self.clusters = self.add_weight(
            shape=(self.n_clusters, input_shape[1]),
            initializer='glorot_uniform',
            name='clusters'
        )
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

# Build Clustering Model
clustering_layer = ClusteringLayer(n_clusters, weights=[cluster_centers], name='clustering')(encoder)
clustering_model = Model(inputs=input_layer, outputs=clustering_layer)
clustering_model.compile(optimizer='adam', loss='kld')

# Step 4: Define Target Distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(axis=0)
    return (weight.T / weight.sum(axis=1)).T

# Train Clustering Model
max_iter = 20  # Number of iterations for refinement
for epoch in range(max_iter):
    if epoch % 10 == 0:
        print(f"Iteration {epoch}")

    # Predict soft assignments
    q = clustering_model.predict(X_scaled, verbose=0)
    p = target_distribution(q)  # Update target distribution

    # Train on current target distribution
    clustering_model.fit(X_scaled, p, epochs=1, batch_size=32, verbose=0)

# Get Final Cluster Assignments
q_final = clustering_model.predict(X_scaled, verbose=0)
cluster_labels = np.argmax(q_final, axis=1)

print("Cluster Assignments:", cluster_labels)

2025-05-05 21:27:23.196153: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-05 21:27:23.249177: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-05 21:27:24.283983: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-05-05 21:27:58.307814: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-05 21:27:58.388

Epoch 1/100
130/130 [==============================] - 2s 11ms/step - loss: 0.0319
Epoch 2/100
130/130 [==============================] - 1s 11ms/step - loss: 0.0050
Epoch 3/100
130/130 [==============================] - 2s 12ms/step - loss: 0.0048
Epoch 4/100
130/130 [==============================] - 1s 11ms/step - loss: 0.0046
Epoch 5/100
130/130 [==============================] - 1s 11ms/step - loss: 0.0045
Epoch 6/100
130/130 [==============================] - 1s 11ms/step - loss: 0.0044
Epoch 7/100
130/130 [==============================] - 2s 12ms/step - loss: 0.0044
Epoch 8/100
130/130 [==============================] - 1s 11ms/step - loss: 0.0044
Epoch 9/100
130/130 [==============================] - 1s 11ms/step - loss: 0.0043
Epoch 10/100
130/130 [==============================] - 1s 11ms/step - loss: 0.0043
Epoch 11/100
130/130 [==============================] - 1s 11ms/step - loss: 0.0043
Epoch 12/100
130/130 [==============================] - 1s 11ms/step - loss: 0.0043
E

: 

Save cluster assignments

In [ ]:
import pickle

# Pickle the array and save to a file
filename = 'compressed/neural/cluster_labels.pkl'
with open(filename, 'wb') as file:
    pickle.dump(cluster_labels, file)

NameError: name 'cluster_labels' is not defined